In [1]:
import numpy as np
import cv2
import os

In [2]:
##########KNN Code############

def distance(v1, v2):
    #Eucledion
    return np.sqrt(((v1-v2)**2).sum())

In [3]:
def knn(train, test, k=5):
    dist = []
    
    for i in range(train.shape[0]):
        # Get the vector and label
        ix = train[i, :-1]
        iy = train[i, -1]
        # Compute the distance from test point
        d = distance(test, ix)
        dist.append([d, iy])
    #Sort based on distance and get top k
    dk = sorted(dist, key= lambda x: x[0])[:k]
    #Retrieve only the labels
    labels = np.array(dk)[:,-1]
    
    #Get frequencies at each label
    output = np.unique(labels, return_counts=True)
    #Find max frequencies and corresponding label
    index = np.argmax(output[1])
    return output[0][index]
##################################


# initiate web_cam
cap = cv2.VideoCapture(0)

# Face detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

# Data Preparation
class_id = 0 #label for the given file
names = {} # mapping id with name
dataset_path = "./"

face_data = []
labels = []

for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        
        # Create a mapping btw class_id and name
        names[class_id] = fx[:-4]
        
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        # Create labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        
        class_id +=1
        labels.append(target)
        
face_dataset = np.concatenate(face_data, axis = 0)
face_labels = np.concatenate(labels, axis = 0).reshape((-1,1))

train_set = np.concatenate((face_dataset, face_labels), axis = 1)
print (train_set.shape)

# Testing

while True:
    ret, frame = cap.read()
    
    if ret==False:
        continue
        
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
    if len(faces)==0:
        continue

    # Pick the last face (because it has the largest area)
    for face in faces:
        # draw bounding box or rectangle 
        
        x, y, w, h = face  
                
        # Extract (crop out the rrequired face) : region of interest
        offset = 10
        face_section = gray_frame[y-offset:y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section, (100,100))
        
        #predict
        out = knn(train_set, face_section.flatten(), k=5)
        
        #Display the output on the screen
        pred_name = names[int(out)]
        cv2.putText(gray_frame, pred_name, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,56,2), 2, cv2.LINE_AA)
        cv2.rectangle(gray_frame, (x,y), (x+w,y+h), (255,255,0), 2)

        
    #cv2.imshow("Frame", frame)
    cv2.imshow("gray_frame", gray_frame)
    
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
        
        
cap.release()
cv2.destroyAllWindows()

(145, 10001)
